# EDUFACE

-    this is a facial recognition system that will be integrated together with an LMS system to record student's attendance using facial recognition technology integrated.

### INSTALLING THE DEPENDENCIES.

In [7]:
!pip install tensorflow opencv-python matplotlib



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### IMPORTING THE DEPENDENCIES.

- we will install the standard dependencies first then install the tensor-flow dependencies later.

In [8]:
#isntalling standard dependencies.

import cv2 #used to import open-cv into the noteboook
import os #OS system library that is needed when creating our Directories. it hence abstracts our operating system from our python
import random #used if we are generating new data.
import numpy as np #used when working with mathematical operations.
from matplotlib import pyplot as plt #


In [9]:
#we will now import tensorflow dependencies that will use tensorflow functional API components.
from tensorflow.keras.models import Model #this is one of most important layers since when creating a model we will need this when declaring Model(Inputs=>>> Output=>>) it enables us classify betwen output and input.
#our model will  look like: Model(inputs=[inputImage, verificationImage] output=[1,0]) this is what the model class helps us to do.
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten #model is a whole lot of abig class and we need these specific classes when building our model
#1. layer class is a high level class that helps you define a custom neural network layer
#2. conv2d - allows us to perform convolutions
#3. dense allows us to get a fully connected layer
#4. maxpooling2D allows us to pull our layers together and combine the information that we have through a condition
#5. Input allows us to identofy what we are passing through to our model.
#6. flatten takes the information from  a previous layer and flatten it into a single dimension
import tensorflow as tf

### Set GPU growth.
- here we are going to limit our amount of VRAM given to tensorflow hence you will keep on ending up to "out of memory error"

In [10]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

print("Number of GPUs available:", len(gpus))


Number of GPUs available: 0


### Setting up folder structure.
- we are going to set up the directories for

In [11]:
POS_Path=os.path.join('data', 'positive')
NEG_Path=os.path.join('data', 'negative')
ANC_Path=os.path.join('data', 'anchor')

#the above create the paths for the directories.

#os.path.join joins diferent directroy together to be able to get a full file path and you will be able to get  forward backslash or backward slash depeding on the Os that you are using



In [12]:

#Mke the actual directories specified with the above paths
os.makedirs(POS_Path)
os.makedirs(NEG_Path)
os.makedirs(ANC_Path)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data\\positive'

In [1]:
import tarfile  # For .tgz files
import os

# 1. Path to your LFW .tgz file in your local filesystem
tgz_path = r"C:\Projects\EDUFACE PROJECT\lfw-funneled.tgz"  # 👈 Adjust to the actual path of lfw-funneled.tgz

# 2. Extraction path (where the dataset will be extracted)
extract_path = r"C:\Projects\EDUFACE PROJECT\lfw"  # 👈 Adjust as needed for your local filesystem

# 3. Validate the .tgz file exists
if not os.path.isfile(tgz_path):
    raise FileNotFoundError(f"Error: The file '{tgz_path}' does not exist or is not a file.")

# 4. Extract only if not already extracted
if not os.path.exists(extract_path):
    os.makedirs(extract_path, exist_ok=True)
    try:
        with tarfile.open(tgz_path, 'r:gz') as tar:
            tar.extractall(path=extract_path)
        print("✅ Dataset extracted successfully at:", extract_path)
    except PermissionError:
        raise PermissionError(f"Error: Permission denied when accessing '{tgz_path}' or '{extract_path}'. Check file/folder permissions.")
    except tarfile.ReadError:
        raise ValueError(f"Error: '{tgz_path}' is not a valid .tgz file.")
else:
    print("ℹ️ Dataset already extracted at:", extract_path)

lfw_root = os.path.join(extract_path, "lfw_funneled")

# 5. Quick check of structure
if os.path.exists(lfw_root):
    print("Number of persons in dataset:", len(os.listdir(lfw_root)))
    print("First 5 persons:", os.listdir(lfw_root)[:5])
else:
    print(f"Error: The directory '{lfw_root}' does not exist. Extraction may have failed.")

ℹ️ Dataset already extracted at: C:\Projects\EDUFACE PROJECT\lfw
Number of persons in dataset: 0
First 5 persons: []


### moving the lfw data into the data/negative data repository

In [ ]:

for directory in os.listdir('lfw'):
  for file in os.listdir(os.path.join('lfw', directory)):
    EX_PATH=os.path.join('lfw', directory, file)
    NEW_PATH=os.path.join(NEG_Path, file)
    os.replace(EX_PATH, NEW_PATH)



## collect Positves and Anchors.
- we are going to use opencv and we are going to collect those images down nd save them down. the sime of the images to be collected are going to be 250 pixels by 250 pixels.

-   by default, when using webcam, the resolution of the images might be a little different so we should ensure that we are collecting images of that size because the sizes from the training dataset are of the same size.


In [ ]:
import cv2
import numpy as np
import os
import time
import glob
import platform

def find_working_webcam(max_index=10):
    """Try to find a working webcam by testing indices with optional backend."""
    backends = [None]  # Default backend
    if platform.system() == "Windows":
        backends.append(cv2.CAP_DSHOW)  # DirectShow
        backends.append(cv2.CAP_MSMF)   # Windows Media Foundation
    elif platform.system() == "Linux":
        backends.append(cv2.CAP_V4L2)   # V4L2

    for backend in backends:
        for index in range(max_index):
            cap = cv2.VideoCapture(index, backend) if backend else cv2.VideoCapture(index)
            if cap.isOpened():
                ret, frame = cap.read()
                print(f"Testing Index {index}, Backend {backend if backend else 'default'}: Opened={cap.isOpened()}, Frame captured={ret}")
                if ret:
                    print(f"✅ Webcam found at index {index} (backend: {backend if backend else 'default'})")
                    return cap, index
                cap.release()
    return None, None

# Fallback: Use a video file or static image for debugging if no webcam is found
fallback_source = r"C:\Projects\EDUFACE PROJECT\test_video.mp4"  # Replace with a video/image path

# Ask for student details
student_name = input("Enter student name: ").strip().replace(" ", "_")
student_id = input("Enter school ID: ").strip()

# Validate student_id for safe file naming
invalid_chars = '<>:"/\\|?*'
for char in invalid_chars:
    student_id = student_id.replace(char, "_")

# Create dataset folder
dataset_folder = r"C:\Projects\EDUFACE PROJECT\students_faces_dataset"
os.makedirs(dataset_folder, exist_ok=True)

# Create sub-folder for this student
folder_name = os.path.join(dataset_folder, f"{student_name}_{student_id}")
os.makedirs(folder_name, exist_ok=True)

# Determine starting count based on existing files
existing_files = glob.glob(os.path.join(folder_name, f"{student_name}_{student_id}_*.jpg"))
if existing_files:
    counts = []
    for f in existing_files:
        try:
            # Extract number from filename (handle both frame and angle naming)
            basename = os.path.basename(f)
            if 'frame' in basename:
                num = int(basename.split('frame')[1].split('.jpg')[0])
            else:
                # For angle-based naming like "front_1.jpg", "left_2.jpg", etc.
                parts = basename.replace(f"{student_name}_{student_id}_", "").replace(".jpg", "")
                if '_' in parts:
                    num = int(parts.split('_')[-1])
                else:
                    num = 1
            counts.append(num)
        except:
            continue
    count = max(counts) + 1 if counts else 1
else:
    count = 1

# Load multiple Haar Cascades for different face orientations
cascades = {}
cascade_files = {
    'frontal': 'haarcascade_frontalface_default.xml',
    'profile': 'haarcascade_profileface.xml',
    'frontal_alt': 'haarcascade_frontalface_alt.xml',
    'frontal_alt2': 'haarcascade_frontalface_alt2.xml'
}

print("Loading face detection cascades...")
for cascade_name, cascade_file in cascade_files.items():
    cascade_path = cv2.data.haarcascades + cascade_file
    cascade = cv2.CascadeClassifier(cascade_path)
    if not cascade.empty():
        cascades[cascade_name] = cascade
        print(f"✅ Loaded {cascade_name} cascade")
    else:
        print(f"⚠️ Could not load {cascade_name} cascade from {cascade_path}")

if not cascades:
    raise ValueError("Error: Could not load any Haar Cascade classifiers.")

def detect_faces_multi_angle(gray_frame):
    """Detect faces using multiple cascades for different angles"""
    all_faces = []
    detected_types = []
    
    for cascade_name, cascade in cascades.items():
        # Different parameters for different cascade types
        if 'profile' in cascade_name:
            faces = cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=4, minSize=(30, 30))
        else:
            faces = cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))
        
        for face in faces:
            all_faces.append(face)
            detected_types.append(cascade_name)
    
    return all_faces, detected_types

def remove_overlapping_faces(faces, detected_types, overlap_threshold=0.3):
    """Remove overlapping face detections to avoid duplicates"""
    if len(faces) <= 1:
        return faces, detected_types
    
    def calculate_overlap(box1, box2):
        x1, y1, w1, h1 = box1
        x2, y2, w2, h2 = box2
        
        # Calculate intersection area
        x_overlap = max(0, min(x1 + w1, x2 + w2) - max(x1, x2))
        y_overlap = max(0, min(y1 + h1, y2 + h2) - max(y1, y2))
        intersection = x_overlap * y_overlap
        
        # Calculate union area
        area1 = w1 * h1
        area2 = w2 * h2
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0
    
    # Keep track of which faces to keep
    keep = [True] * len(faces)
    
    for i in range(len(faces)):
        if not keep[i]:
            continue
        for j in range(i + 1, len(faces)):
            if not keep[j]:
                continue
            
            overlap = calculate_overlap(faces[i], faces[j])
            if overlap > overlap_threshold:
                # Keep the larger face detection
                area1 = faces[i][2] * faces[i][3]
                area2 = faces[j][2] * faces[j][3]
                if area1 >= area2:
                    keep[j] = False
                else:
                    keep[i] = False
                    break
    
    filtered_faces = [faces[i] for i in range(len(faces)) if keep[i]]
    filtered_types = [detected_types[i] for i in range(len(detected_types)) if keep[i]]
    
    return filtered_faces, filtered_types

# Initialize webcam
cap, webcam_index = find_working_webcam(max_index=10)
if cap is None:
    print("⚠️ No webcam found. Falling back to test source (video/image) for debugging.")
    if os.path.exists(fallback_source):
        cap = cv2.VideoCapture(fallback_source)
        webcam_index = "fallback"
        if not cap.isOpened():
            raise ValueError(f"Error: Could not open fallback source '{fallback_source}'.")
        print(f"Using fallback source: {fallback_source}")
    else:
        raise ValueError(f"Error: No webcam found, and fallback source '{fallback_source}' does not exist.")

print(f"\n👉 Using source: index {webcam_index if webcam_index != 'fallback' else fallback_source}")
print("👉 A video preview window will appear.")
print("👉 Press and hold 'c' to capture images continuously (every 0.2s).")
print("👉 The system will detect faces from multiple angles (front, profile, etc.)")
print("👉 Release 'c' to pause. Press and hold again to continue capturing.")
print("👉 Press 'q' to quit and save all captured images.")
print("⚠️ Images will be cropped to detected faces and saved with angle information.\n")
print("💡 TIP: Move your head to different angles while capturing for a better dataset!")
print("   - Look straight at camera (frontal)")
print("   - Turn left and right (profiles)")
print("   - Slightly tilt up and down\n")

# Flag to control capturing
is_capturing = False

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            if webcam_index == "fallback" and cap.get(cv2.CAP_PROP_POS_FRAMES) >= cap.get(cv2.CAP_PROP_FRAME_COUNT):
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Loop video
                continue
            print("Error: Failed to capture frame from source.")
            break

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces from multiple angles
        faces, face_types = detect_faces_multi_angle(gray)
        
        # Remove overlapping detections
        faces, face_types = remove_overlapping_faces(faces, face_types)

        # Draw rectangles around detected faces with different colors for different types
        colors = {
            'frontal': (0, 255, 0),      # Green for frontal
            'profile': (255, 0, 0),      # Blue for profile
            'frontal_alt': (0, 255, 255), # Yellow for frontal alt
            'frontal_alt2': (255, 0, 255) # Magenta for frontal alt2
        }
        
        for i, (x, y, w, h) in enumerate(faces):
            face_type = face_types[i] if i < len(face_types) else 'frontal'
            color = colors.get(face_type, (0, 255, 0))
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            
            # Add label showing the detection type
            label = face_type.replace('_', ' ').title()
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Add status text
        status_text = f"Faces detected: {len(faces)} | {'CAPTURING' if is_capturing else 'PAUSED'}"
        cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 1)

        # Display the frame
        cv2.imshow("Multi-Angle Face Capture - Press 'c' to capture, 'q' to quit", frame)

        # Check for keypress
        key = cv2.waitKey(1) & 0xFF



#collect the anchor images.
        if key==ord('a'):
            capture_type = 'anchor'
            print("switched to anchor capture")
        elif key == ord('p'):
            capture_type="positive"
            print("supposed to save positive images")

        elif key == ord('c'):
            is_capturing = True
        elif key == 255:  # No key pressed (equivalent to releasing 'c')
            is_capturing = False
        
        elif key == ord('q'):
            print("\n👋 Exiting capture...")
            break

        # Capture and save faces if 'c' is held
        if is_capturing and faces:
            for i, (x, y, w, h) in enumerate(faces):
                face_crop = frame[y:y+h, x:x+w]
                
                # Determine angle/type for filename
                face_type = face_types[i] if i < len(face_types) else 'frontal'
                angle_name = face_type.replace('frontal', 'front').replace('_alt', '').replace('_alt2', '')
                
                # Create filename with angle information
                filename = os.path.join(folder_name, f"{student_name}_{student_id}_{angle_name}_{count}.jpg")
                
                # Resize face crop to standard size (optional, good for training)
                face_resized = cv2.resize(face_crop, (250, 250))
                
                cv2.imwrite(filename, face_resized)
                print(f"✅ Saved {filename} (Type: {face_type})")
                count += 1
            
            time.sleep(0.2)  # Capture every 0.2 seconds to avoid oversaving

finally:
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

print(f"\n👋 Capture session complete. Total images saved: {count - 1}")
print(f"📁 Images saved in: {folder_name}")
print("\n📊 Dataset Summary:")
# Count different types of images saved
angle_counts = {}
for f in glob.glob(os.path.join(folder_name, f"{student_name}_{student_id}_*.jpg")):
    basename = os.path.basename(f)
    if 'front' in basename:
        angle_counts['frontal'] = angle_counts.get('frontal', 0) + 1
    elif 'profile' in basename:
        angle_counts['profile'] = angle_counts.get('profile', 0) + 1
    else:
        angle_counts['other'] = angle_counts.get('other', 0) + 1

for angle, count in angle_counts.items():
    print(f"   - {angle.title()}: {count} images")

    

Loading face detection cascades...
✅ Loaded frontal cascade
✅ Loaded profile cascade
✅ Loaded frontal_alt cascade
✅ Loaded frontal_alt2 cascade
Testing Index 0, Backend default: Opened=True, Frame captured=True
✅ Webcam found at index 0 (backend: default)

👉 Using source: index 0
👉 A video preview window will appear.
👉 Press and hold 'c' to capture images continuously (every 0.2s).
👉 The system will detect faces from multiple angles (front, profile, etc.)
👉 Release 'c' to pause. Press and hold again to continue capturing.
👉 Press 'q' to quit and save all captured images.
⚠️ Images will be cropped to detected faces and saved with angle information.

💡 TIP: Move your head to different angles while capturing for a better dataset!
   - Look straight at camera (frontal)
   - Turn left and right (profiles)
   - Slightly tilt up and down

✅ Saved C:\Projects\EDUFACE PROJECT\students_faces_dataset\dennis_151354\dennis_151354_front_41.jpg (Type: frontal_alt)
✅ Saved C:\Projects\EDUFACE PROJECT\

: 